In [ ]:
pip install -U googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40718 sha256=a0a7053a1a2956d707db216a8b419a0496760bfbda11bc5f58067995ab591612
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [28]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=96e44ba6225ae5fd7152860948004c04a3e246e5a293276f85ea48a1676d4af3
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [1]:
import googlemaps
import requests
import json

# Replace 'YOUR_API_KEY' with your actual API key
api_key = ''

# Create a client instance
gmaps = googlemaps.Client(api_key)

# Define the latitude and longitude coordinates
latitude = -1.2654
longitude = 116.8311

def get_traffic_condition(latitude, longitude):
  # Request traffic information
  traffic_result = gmaps.directions((latitude, longitude), (latitude, longitude + 0.1), mode="driving", departure_time="now", traffic_model="best_guess")
  return traffic_result
  # Extract the traffic information
  duration_without_traffic = traffic_result[0]['legs'][0]['duration']['value']
  duration_in_traffic = traffic_result[0]['legs'][0]['duration_in_traffic']['value']
  traffic_ratio = duration_in_traffic / duration_without_traffic
  if traffic_ratio <= 0.7:  # Current if the ratio of travel time to traffic <= 0.7
    traffic_status = "Lancar"
  elif traffic_ratio <= 1.3:  # Crowded if the ratio of travel time to traffic is between 0.7 and 1.3
    traffic_status = "Ramai"
  elif traffic_ratio <= 2.0:  # Padat jika rasio waktu tempuh dengan lalu lintas antara 1.3 dan 2.0
    traffic_status = "Padat"
  else:  # Macet jika rasio waktu tempuh dengan lalu lintas > 2.0
    traffic_status = "Macet"

  return {
      "duration_without_traffic":duration_without_traffic,
      "duration_with_traffic":duration_in_traffic,
      "traffic_ratio":traffic_ratio,
      "traffic_status":traffic_status
  }

In [11]:
import googlemaps
import requests
import json
import re
from googletrans import Translator


# Replace 'YOUR_API_KEY' with your actual API key
api_key = ''

# Create a client instance
gmaps = googlemaps.Client(api_key)

# Define the latitude and longitude coordinates
latitude_origin = -1.2654
longitude_origin = 116.8311
latitude_destination = -1.2654
longitude_destination = 116.9311

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def capitalize_first_letter(sentence):
    words = sentence.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

def create_route_description(latitude_origin, longitude_origin, latitude_destination, longitude_destination):
  # Request traffic information
  traffic_result = gmaps.directions((latitude_origin, longitude_origin), (latitude_destination, longitude_destination), mode="driving", departure_time="now", traffic_model="best_guess")
  steps = traffic_result[0]['legs'][0]['steps']

  translator = Translator()

  description = ''

  for i, step in enumerate(steps):
    instruction = remove_html_tags(step['html_instructions'])
    distance = step['distance']['text']
    duration = step['duration']['text']

    if i == 0:
      description += f"Mulailah dengan {instruction.lower()} sejauh {distance} selama {duration}.\n"
    elif i == len(steps) - 1:
      description += f"Beloklah ke {instruction.lower()}.\nTujuan akhir akan berada di sebelah kiri."
    else:
      description += f"{capitalize_first_letter(instruction)}. Jarak yang ditempuh sejauh {distance} selama {duration}.\n"

  description = translator.translate(description, dest='id', src='en').text

  return description

In [12]:
print(create_route_description(latitude_origin, longitude_origin, latitude_destination, longitude_destination))

Mulailah Delan Head Southeast Sejauh 8 M Selama 1 mnt.
Belok kiri menuju JL.Telaga Sari II.Jarak Yang Ditempuh SEJAUH 28 M Selama 1 mnt.
Belok kanan ke JL.Telaga Sari II.Jarak Yang Ditempuh SEJAUH 0,4 km Selama 2 menit.
Belok kiri ke JL.Telaga Sari.Jarak Yang Ditempuh SEJAUH 0,2 km Selama 1 menit.
Belok kanan ke JL.Re martadinata.Jarak Yang Ditempuh SEJAUH 0,2 km Selama 1 menit.
Belok kiri ke JL.Kapten Piere Tendean.Jarak Yang Ditempuh SEJAUH 0,3 km Selama 1 menit.
Belok kanan setelah Ikan Hias (di sebelah kiri) melewati Salon Widya (di sebelah kiri dalam 650m).Jarak Yang Ditempuh SEJAUH 1.2 km Selama 3 menit.
Belok kiri setelah LPK Kaltim Balikpapan (di sebelah kiri) melewati KFC Coffee Pasar Baru (di sebelah kiri).Jarak Yang Ditempuh SEJAUH 2.2 km Selama 5 menit.
Sedikit hak untuk tetap di JL.Jenderal Sudirman.Jarak Yang Ditempuh SEJAUH 2.5 km Selama 5 menit.
Lanjutkan ke JL.Marsma R. Iswahyudi.Jarak Yang Ditempuh SEJAUH 4.0 km Selama 6 menit.
Lanjutkan langsung untuk tetap di JL.Mar

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib

# Prediksi Bontang
# Data suhu udara minimum
suhu_udara_rata_rata = np.array([
    [27.6, 27.6, 27.9, 27.8, 27.8, 27.8, 27.9, 28.0, 28.1, 27.8, 27.9, 28.2],
    [29.2, 30.1, 29.8, 29.9, 29.7, 29.0, 29.2, 29.4, 30.3, 29.4, 29.7, 29.2],
    [28.2, 28.6, 28.7, 28.6, 28.4, 27.6, 27.3, 27.5, 27.6, 28.0, 28.1, 27.7]
])

# Mengubah data menjadi bentuk yang sesuai untuk model
X_minimum = np.array([[i+1 for i in range(12)] for _ in range(3)]).flatten().reshape(-1, 1)
y_minimum = suhu_udara_rata_rata.flatten()

# Membuat dan melatih model regresi linier
model_rata_rata = LinearRegression()
model_rata_rata.fit(X_minimum, y_minimum)

# Menyimpan model ke file
joblib.dump(model_rata_rata, "model_prediksi_minimum.pkl")

['model_prediksi_minimum.pkl']

In [13]:
import numpy as np
import joblib

# Memuat model dari file
model_minimum = joblib.load("model_prediksi_minimum.pkl")

# Mengubah bulan menjadi angka untuk representasi numerik
bulan = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4, 'Mei': 5, 'Juni': 6,
    'Juli': 7, 'Agustus': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

# Meminta input bulan dari pengguna
input_bulan = input("Masukkan bulan (Contoh: Januari): ")

# Mengambil angka representasi bulan dari input pengguna
input_bulan_angka = bulan.get(input_bulan)

if input_bulan_angka:
    # Melakukan prediksi suhu udara minimum berdasarkan bulan yang diinputkan
    input_bulan_angka_array = np.array([[input_bulan_angka]])
    prediksi_minimum = model_minimum.predict(input_bulan_angka_array)

    print("Prediksi suhu udara minimum:", prediksi_minimum[0])
else:
    print("Bulan yang Anda inputkan tidak valid.")

Masukkan bulan (Contoh: Januari): Juni
Prediksi suhu udara minimum: 28.4989121989122
